In [105]:
import torch as t
from torch import nn
from torch.nn.functional import relu
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score , confusion_matrix
from torchvision.datasets import MNIST
import idx2numpy as i2n

In [106]:
# data = MNIST(r"C:\Users\LENOVO\Desktop\NLPROADMAP\Block1-M\weektwo\deliverable-4" , download=True)
# data

In [ ]:
import torch as t
import idx2numpy as i2n

# Load training data
xtrain = i2n.convert_from_file("./MNIST/raw/train-images-idx3-ubyte")
ytrain = i2n.convert_from_file("./MNIST/raw/train-labels-idx1-ubyte")

# Preprocess training data
xtrain = xtrain.reshape(60000, 28*28).astype('float32') / 255.0
xtrain = t.tensor(xtrain, dtype=t.float32)
ytrain = t.tensor(ytrain, dtype=t.long)

# Load test data  
xtest = i2n.convert_from_file("./MNIST/raw/t10k-images-idx3-ubyte")
ytest = i2n.convert_from_file("./MNIST/raw/t10k-labels-idx1-ubyte")

# Preprocess test data (same as training)
xtest = xtest.reshape(10000, 28*28).astype('float32') / 255.0
xtest = t.tensor(xtest, dtype=t.float32)
ytest = t.tensor(ytest, dtype=t.long)

print("✅ Data loaded and preprocessed successfully!")
print(f"Training: {xtrain.shape}, {ytrain.shape}")
print(f"Test: {xtest.shape}, {ytest.shape}")

✅ Data loaded and preprocessed successfully!
Training: torch.Size([60000, 784]), torch.Size([60000])
Test: torch.Size([10000, 784]), torch.Size([10000])


<code>MNIST dataset classification using Pytorch</code>

In [108]:
class Model(nn.Module):
    def __init__(self , in_features, hidden_layer_one , hidden_layer_two , out_features):
        super().__init__()
        self.fc1 = nn.Linear(in_features , hidden_layer_one)
        self.fc2 = nn.Linear(hidden_layer_one , hidden_layer_two)
        self.out= nn.Linear(hidden_layer_two , out_features)
    
    def forward(self,x): 
        x = relu(self.fc1(x))
        x = relu(self.fc2(x))
        x = self.out(x)
        return x
        
        
        

In [109]:
model = Model(
    784,128,64,10
)
c = nn.CrossEntropyLoss()
optim = t.optim.Adam(model.parameters() , 0.001)

In [110]:
from torch.utils.data import DataLoader , TensorDataset

train_dataset = TensorDataset(xtrain , ytrain)
train_data_loader = DataLoader(train_dataset , batch_size=64 , shuffle=True)


In [111]:
for i in range(10):
    for batch_idx , (batchx,batchy) in enumerate(train_data_loader):
        y_hat = model(batchx)
        loss = c(y_hat , batchy)
        if i % 100 == 0 :
            print("Loss : " , loss)
        
        optim.zero_grad()
        loss.backward()
        optim.step()

Loss :  tensor(2.2967, grad_fn=<NllLossBackward0>)
Loss :  tensor(2.2932, grad_fn=<NllLossBackward0>)
Loss :  tensor(2.2659, grad_fn=<NllLossBackward0>)
Loss :  tensor(2.2572, grad_fn=<NllLossBackward0>)
Loss :  tensor(2.2627, grad_fn=<NllLossBackward0>)
Loss :  tensor(2.1920, grad_fn=<NllLossBackward0>)
Loss :  tensor(2.2009, grad_fn=<NllLossBackward0>)
Loss :  tensor(2.1196, grad_fn=<NllLossBackward0>)
Loss :  tensor(2.1387, grad_fn=<NllLossBackward0>)
Loss :  tensor(2.0912, grad_fn=<NllLossBackward0>)
Loss :  tensor(2.1082, grad_fn=<NllLossBackward0>)
Loss :  tensor(2.0589, grad_fn=<NllLossBackward0>)
Loss :  tensor(2.0152, grad_fn=<NllLossBackward0>)
Loss :  tensor(1.9971, grad_fn=<NllLossBackward0>)
Loss :  tensor(1.9145, grad_fn=<NllLossBackward0>)
Loss :  tensor(1.9206, grad_fn=<NllLossBackward0>)
Loss :  tensor(1.8693, grad_fn=<NllLossBackward0>)
Loss :  tensor(1.8747, grad_fn=<NllLossBackward0>)
Loss :  tensor(1.6588, grad_fn=<NllLossBackward0>)
Loss :  tensor(1.7357, grad_fn=

In [118]:
with t.no_grad():
    # Get raw predictions (logits)
    y_pred_logits = model(xtest)  # Shape: (10000, 10)
    
    print("Raw logits shape:", y_pred_logits.shape)
    print("\nFirst 5 predictions (logits):")
    # print(y_pred_logits[:5])
    
    # Convert to probabilities
    y_pred_probs = t.softmax(y_pred_logits, dim=1)
    print("\nFirst 5 predictions (probabilities):")
    print(y_pred_probs[:5])
    
    # Get predicted classes
    y_pred_classes = t.argmax(y_pred_logits, dim=1)
    print("\nFirst 10 predicted classes:")
    print(y_pred_classes[:10])
    
    # Get true classes
    print("\nFirst 10 true classes:")
    print(ytest[:10])
    
    # Calculate accuracy
    accuracy = (y_pred_classes == ytest).float().mean()
    print(f"\nTest Accuracy: {accuracy.item()*100:.2f}%")
    cm = confusion_matrix(ytest , y_pred_classes)
    print(cm)

Raw logits shape: torch.Size([10000, 10])

First 5 predictions (logits):

First 5 predictions (probabilities):
tensor([[1.5738e-13, 6.3942e-10, 7.9947e-10, 2.8957e-07, 1.1578e-14, 1.4451e-09,
         7.0406e-20, 1.0000e+00, 4.6505e-10, 9.3880e-09],
        [2.8429e-11, 7.8095e-08, 1.0000e+00, 2.0476e-09, 2.5915e-22, 4.5786e-10,
         9.5581e-11, 6.0107e-16, 1.5391e-09, 9.0272e-23],
        [2.1486e-07, 9.9939e-01, 1.3669e-04, 2.1558e-06, 4.2452e-06, 1.0039e-05,
         2.0577e-05, 4.0993e-04, 2.2321e-05, 7.6352e-11],
        [9.9999e-01, 1.2755e-09, 5.0570e-07, 9.5871e-11, 4.2709e-11, 1.5491e-07,
         5.1425e-06, 1.1236e-08, 5.8347e-10, 5.2420e-08],
        [1.4176e-07, 4.4497e-08, 3.4324e-08, 2.1902e-10, 9.9929e-01, 6.0254e-09,
         3.7085e-08, 5.0512e-05, 1.0499e-07, 6.5805e-04]])

First 10 predicted classes:
tensor([7, 2, 1, 0, 4, 1, 4, 9, 5, 9])

First 10 true classes:
tensor([7, 2, 1, 0, 4, 1, 4, 9, 5, 9])

Test Accuracy: 97.64%
[[ 961    0    2    2    2    5    3   